# Iterative Schema with TFX and ML Metadata

### Setup

In [ ]:
pip install tfx

In [3]:
import tensorflow as tf
import tensorflow_data_validation as tfdv

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.v1.dsl import Importer
# from tfx.components import Importer
from tfx.types import standard_artifacts

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_metadata.proto.v0 import schema_pb2

import os
import pprint
pp = pprint.PrettyPrinter()
